In [1]:
# 準備工作

from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import datetime
import re
import sqlite3
import os
import json
import pprint

db_name = "../data/21_century.sqlite"

In [2]:
# 從db_name取出資料

with sqlite3.connect(db_name) as conn_retrieve:
    
    c_rt = conn_retrieve.cursor()
    qryString = "SELECT * from rental;"
    c_rt.execute(qryString)
    retrieved_list = c_rt.fetchall()

c_rt.close()

In [3]:
# 定義 recover_quot() 方法 (recover quotation marks)
# 將字串中連續兩個的單引號跟雙引號復原為
# 單一的單引號跟雙引號

def recover_quot(string):
    string = string.replace("\"\"","\"")
    string = string.replace("\'\'","\'")
    return string

In [4]:
# 定義 remove_dust() 方法
# 清理一個字串中所有的 \r, \n, \t, \xa0 符號
# 並將頭尾的空格去掉

def remove_dust(string):
    string = string.replace("\r","")
    string = string.replace("\n","")
    string = string.replace("\t","")
    string = string.replace("\xa0","")
    string = string.strip()
    return string

In [422]:
# 定義 remove_dirt() 方法
# 用於移除'環境介紹'中的雜物

def remove_dirt(text):
    
    replace_set = [('：',':'),('︰',':'),(', ',','),
                  ('、 ','、'),('※',''),('\xa0',''),
                  ('ˊ',''),('\n+?',''),
                  ('備註:',''),('查詢主頁動態',''),
                  ('歡迎房客加入line或微信 ID','網路社群ID'),
                  (' ','')]

    for pair in replace_set:
        text = re.sub(pair[0],pair[1],text).strip()
        
    return text

In [423]:
# 定義 remove_multi_space 方法
# 清理多空格

def remove_multi_space(string):
    while "  " in string:
        string = string.replace("  ","")
    return string

In [424]:
# 判斷一個字串中分號是否在換行符號前面
# 或不存在

def is_col_before_break(string):
    try:
        first_colon_pos = string.index(':')
        try:
            first_break_pos = string.index('\n')
            return first_break_pos > first_colon_pos
        except:
            return True 
    except:
        return False

In [425]:
# 定義 get_section_data() 方法
# 從每個章節中抽出資料

def get_section_data():
    
    section_data = {}
    sections = [each.text for each in soup.find_all('h3')]
    
    for section_name in sections:

        section = soup.find_all('h3',text=section_name)[0]
        section = section.find_parent()

        if not section_name == "環境介紹":

            section_key_tags = section.select('h6')
            section_keys = [each.text for each in section_key_tags]
            section_values = [each.find_next().text for each in section_key_tags]

            # 清理不必要的字串
        
            section_keys = [remove_dust(each) for each in section_keys]
            section_values = [remove_dust(each) for each in section_values]
            
        else:

            section_keys = ['環境介紹']            
            section_values = section.select('div > div')
            
            if len(section_values) == 0:
                section_values = [""]
        
        
        # 塞值到 section_dict 中

        section_data.update(dict(zip(section_keys, section_values)))   
    
    # 回傳
    
    return section_data

In [426]:
# 建立 get_data_dict 方法
# 給代表網頁內容的soup物件
# 取得每頁的資料 並以dictionary形式傳回

def get_data_dict(soup):

    # 內頁標籤1(如"租屋")
    tag01 = soup.select('div[id="breadCrumb"] > ul > li')[1].text

    # 內頁標籤2(如"租屋列表")
    tag02 = soup.select('div[id="breadCrumb"] > ul > li')[2].text

    # 內頁標籤3(如"光復南路優質地段B1(租10)")
    tag03 = soup.select('div[id="breadCrumb"] > ul > li')[3].text


    # 仲介資訊
    
    try:
 
        agent_tags = soup.find_all('div',class_=re.compile("column staff"))
        agent_names = [each.select_one('h4').text for each in agent_tags]

        # 證號

        agent_licenses = [remove_dust(each.find('div',class_="LicenseNo").text) for each in agent_tags]

        # 電話

        agent_contact_tags = soup.select('div[name="agentContactInfo"]')
        agent_phones = []

        # 先找出每個仲介的所有電話(可能不只1個) 個別存成list
        # 但因為select後每個仍為含有tag的list
        # 所以得再將每個list裡面的tag挑出來 取出真正的電話資料後再存回

        for each_list in agent_contact_tags:
            each_agent_phones = []
            each_list = each_list.select('span[class="mobileInfo"] > a')

            for each in each_list:
                each_agent_phones.append(each.text)

            agent_phones.append(each_agent_phones)


        # email
        # 跟類似處理電話的方式處理

        agent_emails_tags = [each.find_all('a',href=re.compile("mailto:")) for each in agent_contact_tags]
        agent_emails = []

        for each_list in agent_emails_tags:
            each_agent_emails = []  

            for each in each_list:
                each_agent_emails.append(each.text)

            agent_emails.append(each_agent_emails)




        # 整合仲介資料

        agent_data = {}

        # each_agent_data = {
        #     '證號': agent_licenses,
        #     '電話': agent_phones,
        #     'email': agent_emails        
        # }

        for j in range(0,len(agent_names)):

            agent_data.update(
                {agent_names[j]: {    # 代表仲介姓名
                    '證號': agent_licenses[j],
                    '電話': agent_phones[j],
                    'email': agent_emails[j]  
                }
            })

    except:
        
        agent_data = "NULL"
    
        
    # 房東資訊

    try:
        
        section = soup.find('div',class_=re.compile("column owner"))
        landlord_name = section.select_one('h3').text
        landlord_details = section.select_one('span').text

        landlord_data = {
            'name': landlord_name,
            'details': landlord_details
        }
    
    except:
        
        landlord_data = "NULL"
    

    # 加盟店

    try:

        branch_tags = soup.find('div',class_=re.compile("column store"))
        branch_name = branch_tags.select_one('h4').text
        branch_co_name = branch_tags.select('ul > li')[0].text
        branch_address = branch_tags.select('ul > li')[1].text
        branch_phone = branch_tags.select('ul > li')[2].text
        branch_fax = branch_tags.select('ul > li')[3].text

        branch_data = {
            '名稱': branch_address,
            '分行公司': branch_co_name,  
            '地址': branch_address,
            '電話': branch_phone,
            '傳真': branch_fax
        }

        # 整理加盟店資訊

        branch_data = {key:remove_dust(branch_dict[key]).strip() for key in store_dict }
        
    except:
        
        branch_data = "NULL"
    
    




    # 存放單筆地點資料用的dictionary
    data_dict = {
        '標籤01': tag01,
        '標籤02': tag02,
        '地點名稱': tag03,
        '仲介': agent_data,
        '房東': landlord_data,
        '分行': branch_data
    }


    # 跟其他dictionary合併

    data_dict.update(get_section_data())


    # 傳回結果

    return data_dict

In [427]:
# 跑迴圈 清洗資料
# 並將結果暫存到一個大的dictionary中

index_set = range(0,10)    # 測試用
dict_all = {}

for i in index_set:

    # 將內頁內容轉成 BeautifulSoup 物件

    page_source = retrieved_list[i][1]
    page_source = recover_quot(page_source) # 還原引號
    soup = BeautifulSoup(page_source,'html.parser')

    # 網址
    # posting_id 為上架ID

    url = retrieved_list[i][0]
    posting_id = url.split("/")[-1]

#     print(i)
#     print(posting_id)
    
    # 清理

    data_dict = get_data_dict(soup)
    data_dict.update({'url':url})

#     pprint.pprint(data_dict)

    # 暫存

    dict_all.update({posting_id: data_dict})

In [439]:
# 定義 desc_as_dict 方法
# 將'環境介紹'的內容拆成dictionary

def desc_as_dict(desc):

    # 第1步驟
    # 從tag中取出文字並存放在list中

    list_1 = [each.text+"\r" for each in desc.select('p')]

    # 第2步驟
    # 將各項目細分後裝到新的list中

    i = 0
    list_2 = []

    for each in list_1:
        each = remove_dirt(each)                    # 清理
        each = re.split('\r|●|◆',str(each))         # 細分內文
        for each2 in each:
            black_list = ['',' ','\xa0','●','◆']    # 排除異樣字元
            if each2 not in black_list:

                if ":" not in each2:                # 加上以後的key值
                    i += 1
                    each2 = "說明" + str(i) + ":" + each2
                list_2.append(each2)                # 存入list_2

    # 第3步驟
    # 轉為dictionary

    desc_dict = {}

    for each in list_2:
        pair = each.split(":")
        desc_dict.update({pair[0]:pair[1]})

        
    pprint.pprint(list_1)
    pprint.pprint(list_2)
#     pprint.pprint(desc_dict)
        
        
        
    # 第3步驟
    # 傳回值
        
    return desc_dict



In [442]:
each_id = '13108'

desc = dict_all[each_id]['環境介紹']
desc_dict = desc_as_dict(desc)
print(each_id)
pprint.pprint(desc_dict)

['花博陽光電梯大套房0926-986801\r',
 '中山區松江路6樓 (臨民族東路)\r',
 '●租金：18000元/月(誠租可談)\r',
 '●連絡人：0926-986801藍先生或李先生\r',
 '●押金：二個月\r',
 '●管理費：1350元/月\r',
 '●格局：1大房1廳1衛1廚 +陽台\r',
 '●租期：最短一年\r',
 '●坪數︰22坪,新整理\r',
 '●開伙：可～流理台廚具\r',
 '●樓層︰6F/6F\n含第四台與網路\r',
 '●養寵物：可，不影響社區住戶\r',
 '●用途︰亮麗獨立ˊ住家套房\r',
 '●型態︰電梯大樓\r',
 '●備註：外籍主管,雅痞族, 百年新婚\r',
 '●車位：停車位樓下可另租(租金為4000~4500元)\r',
 '\xa0\r',
 '歡迎房客加入line或微信 ID：lan395619 查詢主頁動態\r',
 '\xa0特色說明：適商辦人士雙併電梯，24H管理，安全無煩惱、監視系統\r',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n'
 '邊間有窗採光空氣棒，垃圾集中區(免分類免購環保袋) 屋新剛整\xa0\xa0 \r',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 理、含傢俱立可進駐、生活機能樣樣齊全。VIP運動休閒設備免費使\r',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 用,熱水器、流理台冷氣、另有車位可承租、含第四台與網路費用\r',
 '\xa0生活機能：百貨公司、 大賣場、 便利商店、 傳統市場、 夜市、 近公園綠地、 \r',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n近學校、 近醫療機構、 近警察局、公車站牌、捷運、\r',
 '\xa0\r',
 '※請注意：本廣告是代房東刊登,房客安心,過程完全透明,看屋無須支付任何費用.歡迎查詢\r',
 '\xa0\r']
['說明1:花博陽光電梯大套房0926-986801',
 '說明2:中山區松江路6樓(臨民族東路)',
 '租金:18000元/月(誠租可談)',
 '連絡人:0926-986801藍先生或李先生',
 '押金:二個月',
 '管理費:1350元/月'

In [441]:
# 測試 desc_as_dict()

for each_id in dict_all:
    desc = dict_all[each_id]['環境介紹']
    desc_dict = desc_as_dict(desc)
    print(each_id)
    pprint.pprint(desc_dict)

[]
[]
13645
{}
[]
[]
13874
{}
[]
[]
11876
{}
['士林區重慶北路四段4樓(臨重陽橋)\r',
 '●連絡人：0926-986801王先生或藍先生\r',
 '●租金：\n\xa04樓租35000元,約50坪, 3房2廳2衛1廚3陽\r',
 '●坪數︰\n約50坪,使用空間超大.邊間亮麗\r',
 '●樓層︰4F/12F電梯華廈\r',
 '●用途︰整層辦公住家皆宜\r',
 '●備註︰坐北朝南 臨路停車方便\r',
 '備註︰歡迎房客加入line或微信 ID：lan395619 查詢主頁動態\r',
 '\xa0◆租金不含這些～水費、電費、第四台、網路、清潔費\r',
 '◆能養寵物\xa0 ◆可以開伙\xa0\n'
 '◆不能短租(一年為準)\xa0 ◆能設籍或登記(稅金須外加)\xa0 ◆基本電器設備：熱水器、冷氣、瓦斯爐\xa0 '
 '◆除固定裝修之設備外,未註明依現況交屋出租\xa0 ◆押金為二個月\r',
 '\xa0近重慶湖東路口公車站； 社子市場公車站店面\xa0,住辦\xa0,兩相宜\n'
 '上重慶北路交流道一分鐘,租金可議,正重慶北路上,邊間3面採光\n'
 '近高速公路交通方便,近百齡橋\xa0\xa0重陽大橋\xa0\xa0往來社子\xa0\xa0士林\xa0\xa0三重十分方便\n'
 '重慶北路正馬路上\xa0\xa0離中山高速公路也不遠\xa0\xa0\xa0交通便利,住辦大樓\xa0\xa0警衛管理\xa0\xa0\xa0\xa0'
 '陽台,生活機能完善、環境良好，安靜\xa0\xa0採光\xa0\xa0空間大\xa0\xa0使用多\xa0\xa0便宜\xa0\xa0要租要快\r',
 '※請注意：本廣告是代房東刊登,房客安心,過程完全透明,看屋無須支付費用.歡迎查詢\r',
 '\xa0\r']
['說明1:士林區重慶北路四段4樓(臨重陽橋)',
 '連絡人:0926-986801王先生或藍先生',
 '租金:4樓租35000元,約50坪,3房2廳2衛1廚3陽',
 '坪數:約50坪,使用空間超大.邊間亮麗',
 '樓層:4F/12F電梯華廈',
 '用途:整層辦公住家皆宜',
 '說明2:坐北朝南臨路停車方便',
 '網路社群ID:lan395619',
 '說明3:租金不含這些～